# Espresso Shot Data Analysis

## Imports

In [ ]:
import datetime
import glob
import io
import json
import time

from ipywidgets import widgets
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pytz
import seaborn as sns

%matplotlib widget

## Data loading

In [ ]:
df_list = []
for file_path in glob.glob('data/*.json'):
  with open(file_path, 'r') as f:
    data = json.load(f)
  df_list.append(pd.DataFrame({
      'Time (s)': data['time'] + data['time'],
      'Temperature (°C)': data['basket_temperature'] + data['group_temperature'],
      'Location': ['Puck'] * len(data['time']) + ['Group'] * len(data['time']),
      'Date': datetime.datetime.fromtimestamp(
          data['posix time']
      ).astimezone(
          pytz.timezone('US/Eastern')
      ).isoformat(' ', timespec='seconds')
  }))
df = pd.concat(df_list)

available_data = sorted(list(set(df['Date'])))
selected_data = widgets.SelectMultiple(
    options=available_data,
    value=available_data[:1],
    description='Data:',
    disabled=False
)

display(selected_data)

## Data plotting

In [ ]:
data = df.copy().sort_values('Date')

# Discretize time to the nearest integer to get a set of temperature
# measurements over the span of every second. This allows us to compute
# 95% confidence intervals over these values.
data['Time (s)'] = data['Time (s)'].apply(np.around)

fig, ax = plt.subplots()
line = sns.lineplot(
    ax=ax,
    data=data[data['Date'].isin(selected_data.value)],
    x='Time (s)',
    y='Temperature (°C)',
    hue='Date',
    style='Location',
    ci=95)
fig.show()